# NCAR Innovator Program Project

### Notebook by: Maria J. Molina (NCAR/University of Maryland) and Jessica Moulite (Howard University)

Importing packages

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

### Functions

In [2]:
def cel_to_far(tmp):
    """
    Convert celsius to farenheit
    """
    newtmp = []
    for i in tmp:
        if i == 99.9:
            newtmp.append(np.nan)
        if i != 99.9:
            newtmp.append((i * (9/5)) + 32)
    return np.array(newtmp)


def cel_to_nan(tmp):
    """
    Change 9999 to np.nan
    """
    newtmp = []
    for i in tmp:
        if i == 99.9:
            newtmp.append(np.nan)
        if i != 99.9:
            newtmp.append(i)
    return np.array(newtmp)

### NOAA Data

Downloaded data from: https://www.ncei.noaa.gov/access/search/data-search/global-hourly

Data documentation: https://www.ncei.noaa.gov/metadata/geoportal/rest/metadata/item/gov.noaa.ncdc:C00532/html#

In [3]:
# importing MIA airport data (ASOS station from NOAA/FAA)
MIA_temp = pd.read_csv('/Users/molina/Desktop/72202012839.csv')

time_date = []
temperature_C = []

for j, i in zip(MIA_temp['DATE'].values, MIA_temp['TMP'].values.astype(str)):
    time_date.append(j) 
    # convert the string format to decimal format (originally in METAR code)
    temperature_C.append(float(i.split(',')[0][2:-1]+'.'+i.split(',')[0][-1]))
    
time_date = np.array(time_date)
temperature_C = np.array(temperature_C)

temperature_F = cel_to_far(temperature_C)
temperature_C = cel_to_nan(temperature_C)

/Users/molina/opt/anaconda3/envs/torchy/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (48) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# making into dataframe for easier manipulation

# initialize data of lists.
data = {'Datetime': pd.to_datetime(time_date),
        'Temperature_C': temperature_C,
        'Temperature_F': temperature_F,
       }

# Create DataFrame and quality control
df = pd.DataFrame(data)
df = df.dropna()  # drop any nan values
df = df.drop_duplicates(keep='first') # drop any duplicate rows
df = df.drop(8168) # drop repeat datetime with too warm second observation
df = df[(df['Datetime']>='2019-06-08')&(df['Datetime']<='2019-09-01')] # subset the data using dates of interest

### Shading Dade Data

In [5]:
SD_temp = pd.read_excel('/Users/molina/Desktop/ShadingDadeData.xlsx')
SD_temp = SD_temp[(SD_temp['Datetime']>='2019-06-08')&(SD_temp['Datetime']<='2019-09-01')] 

### Create new variables

In [6]:
# Daily temperature maximum for Shading Dade
tmax_SD_daily = SD_temp.set_index('Datetime').resample('1D').max()

In [7]:
# Daily temperature maximum for Shading Dade
tmin_SD_daily = SD_temp.set_index('Datetime').resample('1D').min()

In [8]:
# Daily temperature maximum for NOAA (KMIA)
tmax_NOAA_daily = df.set_index('Datetime').resample('1D').max()

In [9]:
# Daily temperature minimum for NOAA (KMIA)
tmin_NOAA_daily = df.set_index('Datetime').resample('1D').min()

In [10]:
# corresponding date/time for shading dade daily temperature maximum
tmax_SD_daily_datetime = SD_temp.set_index('Datetime').resample('1D').agg(
    lambda x : np.nan if x.count() == 0 else x.idxmax()
)['Temperature (C)']

In [11]:
# corresponding date/time for shading dade daily temperature minimum
tmin_SD_daily_datetime = SD_temp.set_index('Datetime').resample('1D').agg(
    lambda x : np.nan if x.count() == 0 else x.idxmin()
)['Temperature (C)']

In [12]:
# corresponding date/time for noaa daily temperature maximum
tmax_NOAA_daily_datetime = df.set_index('Datetime').resample('1D').agg(
    lambda x : np.nan if x.count() == 0 else x.idxmax()
)['Temperature_C']

In [13]:
# corresponding date/time for noaa daily temperature minimum
tmin_NOAA_daily_datetime = df.set_index('Datetime').resample('1D').agg(
    lambda x : np.nan if x.count() == 0 else x.idxmin()
)['Temperature_C']

In [14]:
# the NOAA temperature during the (same time as the) daily maximum temperature of Shading Dade

NOAA_temp_during_SD_tmax_C = []
NOAA_temp_during_SD_tmax_F = []

for i in tmax_SD_daily_datetime.values:
    
    NOAA_temp_during_SD_tmax_C.append(
        df.iloc[df.set_index('Datetime').index.get_loc(i, method='nearest')]['Temperature_C'])
    
    NOAA_temp_during_SD_tmax_F.append(
        df.iloc[df.set_index('Datetime').index.get_loc(i, method='nearest')]['Temperature_F'])
    
NOAA_temp_during_SD_tmax_C = np.array(NOAA_temp_during_SD_tmax_C)
NOAA_temp_during_SD_tmax_F = np.array(NOAA_temp_during_SD_tmax_F)

In [15]:
# the NOAA temperature during the (same time as the) daily minimum temperature of Shading Dade

NOAA_temp_during_SD_tmin_C = []
NOAA_temp_during_SD_tmin_F = []

for i in tmin_SD_daily_datetime.values:
    
    NOAA_temp_during_SD_tmin_C.append(
        df.iloc[df.set_index('Datetime').index.get_loc(i, method='nearest')]['Temperature_C'])
    
    NOAA_temp_during_SD_tmin_F.append(
        df.iloc[df.set_index('Datetime').index.get_loc(i, method='nearest')]['Temperature_F'])
    
NOAA_temp_during_SD_tmin_C = np.array(NOAA_temp_during_SD_tmin_C)
NOAA_temp_during_SD_tmin_F = np.array(NOAA_temp_during_SD_tmin_F)

In [16]:
# the Shading Dade temperature during the (same time as the) daily maximum temperature of NOAA KMIA

SD_temp_during_NOAA_tmax_C = []
SD_temp_during_NOAA_tmax_F = []

for i in tmax_NOAA_daily_datetime.values:
    
    SD_temp_during_NOAA_tmax_C.append(
        SD_temp.iloc[SD_temp.set_index('Datetime').index.get_loc(i, method='nearest')]['Temperature (C)'])
    
    SD_temp_during_NOAA_tmax_F.append(
        SD_temp.iloc[SD_temp.set_index('Datetime').index.get_loc(i, method='nearest')]['Temperature (F)'])
    
SD_temp_during_NOAA_tmax_C = np.array(SD_temp_during_NOAA_tmax_C)
SD_temp_during_NOAA_tmax_F = np.array(SD_temp_during_NOAA_tmax_F)

In [17]:
# the Shading Dade temperature during the (same time as the) daily minimum temperature of NOAA KMIA

SD_temp_during_NOAA_tmin_C = []
SD_temp_during_NOAA_tmin_F = []

for i in tmin_NOAA_daily_datetime.values:
    
    SD_temp_during_NOAA_tmin_C.append(
        SD_temp.iloc[SD_temp.set_index('Datetime').index.get_loc(i, method='nearest')]['Temperature (C)'])
    
    SD_temp_during_NOAA_tmin_F.append(
        SD_temp.iloc[SD_temp.set_index('Datetime').index.get_loc(i, method='nearest')]['Temperature (F)'])
    
SD_temp_during_NOAA_tmin_C = np.array(SD_temp_during_NOAA_tmin_C)
SD_temp_during_NOAA_tmin_F = np.array(SD_temp_during_NOAA_tmin_F)

### Create single CSV file with data

In [18]:
data = {'ShadingDade_Temperature_Daily_Max (C)': tmax_SD_daily['Temperature (C)'],
        'ShadingDade_Temperature_Daily_Min (C)': tmin_SD_daily['Temperature (C)'],
        'NOAA_KMIA_Temperature_Daily_Max (C)': tmax_NOAA_daily['Temperature_C'],
        'NOAA_KMIA_Temperature_Daily_Min (C)': tmin_NOAA_daily['Temperature_C'],
        
        'ShadingDade_Temperature_Daily_Max (F)': tmax_SD_daily['Temperature (F)'],
        'ShadingDade_Temperature_Daily_Min (F)': tmin_SD_daily['Temperature (F)'],
        'NOAA_KMIA_Temperature_Daily_Max (F)': tmax_NOAA_daily['Temperature_F'],
        'NOAA_KMIA_Temperature_Daily_Min (F)': tmin_NOAA_daily['Temperature_F'],
        
        'Corresponding_Datetime_ShadingDade_DailyMax': tmax_SD_daily_datetime.values,
        'Corresponding_Datetime_ShadingDade_DailyMin': tmin_SD_daily_datetime.values,
        'Corresponding_Datetime_NOAA_KMIA_DailyMax': tmax_NOAA_daily_datetime.values,
        'Corresponding_Datetime_NOAA_KMIA_DailyMin': tmin_NOAA_daily_datetime.values,
        
        'NOAA_Corresponding_Temperature_During_ShadingDade_DailyMax (C)': NOAA_temp_during_SD_tmax_C,
        'NOAA_Corresponding_Temperature_During_ShadingDade_DailyMin (C)': NOAA_temp_during_SD_tmin_C,
        'ShadingDade_Corresponding_Temperature_During_NOAA_DailyMax (C)': SD_temp_during_NOAA_tmax_C,
        'ShadingDade_Corresponding_Temperature_During_NOAA_DailyMin (C)': SD_temp_during_NOAA_tmin_C,
        
        'NOAA_Corresponding_Temperature_During_ShadingDade_DailyMax (F)': NOAA_temp_during_SD_tmax_F,
        'NOAA_Corresponding_Temperature_During_ShadingDade_DailyMin (F)': NOAA_temp_during_SD_tmin_F,
        'ShadingDade_Corresponding_Temperature_During_NOAA_DailyMax (F)': SD_temp_during_NOAA_tmax_F,
        'ShadingDade_Corresponding_Temperature_During_NOAA_DailyMin (F)': SD_temp_during_NOAA_tmin_F,
       }

In [19]:
df_FINAL = pd.DataFrame(data)

In [20]:
df_FINAL

,ShadingDade_Temperature_Daily_Max (C),ShadingDade_Temperature_Daily_Min (C),NOAA_KMIA_Temperature_Daily_Max (C),NOAA_KMIA_Temperature_Daily_Min (C),ShadingDade_Temperature_Daily_Max (F),ShadingDade_Temperature_Daily_Min (F),NOAA_KMIA_Temperature_Daily_Max (F),NOAA_KMIA_Temperature_Daily_Min (F),Corresponding_Datetime_ShadingDade_DailyMax,Corresponding_Datetime_ShadingDade_DailyMin,Corresponding_Datetime_NOAA_KMIA_DailyMax,Corresponding_Datetime_NOAA_KMIA_DailyMin,NOAA_Corresponding_Temperature_During_ShadingDade_DailyMax (C),NOAA_Corresponding_Temperature_During_ShadingDade_DailyMin (C),ShadingDade_Corresponding_Temperature_During_NOAA_DailyMax (C),ShadingDade_Corresponding_Temperature_During_NOAA_DailyMin (C),NOAA_Corresponding_Temperature_During_ShadingDade_DailyMax (F),NOAA_Corresponding_Temperature_During_ShadingDade_DailyMin (F),ShadingDade_Corresponding_Temperature_During_NOAA_DailyMax (F),ShadingDade_Corresponding_Temperature_During_NOAA_DailyMin (F)
Datetime,,,,,,,,,,,,,,,,,,,,
2019-06-08,34.875,23.000,32.8,24.0,94.775,73.400,91.04,75.20,2019-06-08 12:00:00.000,2019-06-08 10:59:59.712,2019-06-08 15:53:00,2019-06-08 19:51:00,28.3,27.2,23.625,25.000,82.94,80.96,74.525,77.000
2019-06-09,33.750,25.125,31.1,25.0,92.750,77.225,87.98,77.00,2019-06-09 12:00:00.000,2019-06-09 21:00:00.000,2019-06-09 15:53:00,2019-06-09 18:31:00,27.8,27.2,25.625,28.250,82.04,80.96,78.125,82.850
2019-06-10,38.500,23.250,31.7,23.9,101.300,73.850,89.06,75.02,2019-06-10 16:00:00.288,2019-06-10 07:00:00.288,2019-06-10 19:53:00,2019-06-10 11:36:00,25.6,26.7,27.375,25.250,78.08,80.06,81.275,77.450
2019-06-11,40.750,24.250,32.2,26.0,105.350,75.650,89.96,78.80,2019-06-11 12:00:00.000,2019-06-11 03:00:00.000,2019-06-11 16:53:00,2019-06-11 23:53:00,28.3,27.8,36.000,26.000,82.94,82.04,96.800,78.800
2019-06-12,40.250,24.875,32.8,25.6,104.450,76.775,91.04,78.08,2019-06-12 10:59:59.712,2019-06-12 04:00:00.288,2019-06-12 19:18:00,2019-06-12 00:09:00,27.8,26.7,28.625,26.000,82.04,80.06,83.525,78.800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-28,40.750,24.625,35.6,25.6,105.350,76.325,96.08,78.08,2019-08-28 13:59:59.712,2019-08-28 06:00:00.000,2019-08-28 19:53:00,2019-08-28 21:53:00,31.1,28.3,25.750,25.625,87.98,82.94,78.350,78.125
2019-08-29,41.375,24.750,31.7,26.7,106.475,76.550,89.06,80.06,2019-08-29 13:00:00.288,2019-08-29 04:00:00.288,2019-08-29 14:53:00,2019-08-29 00:53:00,30.0,27.2,29.375,25.875,86.00,80.96,84.875,78.575
2019-08-30,35.000,23.875,32.2,25.0,95.000,74.975,89.96,77.00,2019-08-30 13:59:59.712,2019-08-30 07:59:59.712,2019-08-30 19:53:00,2019-08-30 11:02:00,27.2,26.7,29.750,30.250,80.96,80.06,85.550,86.450


In [21]:
df_FINAL.to_csv('/Users/molina/Desktop/ShadingDade_NOAA_KMIA_Temperature.csv')

### To do next

(done) Inquire about the METAR comma with the temperature data (Described on page 5: https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf)

(done) Inquire about the ASOS ``source`` column (Description on page 11: https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf)

(1) Compute differences between columns and resave csv

(2) Add ``ERA5`` reanalysis temperature data and resave csv

(3) Redo analysis for other cities: Bronx (NY), Baltimore (MD), Houston (TX), Washington DC, Everglades (FL)